# Main Insights

Revised research questions:
1. How does the frequency with which agencies collect samples affect the number of HPAI cases detected?
    - Which agencies collect samples most frequently?
    - When do agencies collect samples most frequently (time of year)?
    - Where do agencies collect samples most frequently (geographic region)?
2. Is there a correlation between geographic region and season, and number of HPAI cases?
    - How does agency collection frequency influence this correlation?
3. What is the forecast for the number of HPAI cases in each region over the next 6 months?
    - With consideration for agency collection frequency and its impact on case detection.

The goal of this notebook is to answer these questions through statistical analysis.

## Load Data

In [41]:
import pandas as pd

dataset_path = '../data/HPAI Detections in Wild Birds.csv'
hpai_data = pd.read_csv(dataset_path)
hpai_data.head(n=3)

,State,County,Collection Date,Date Detected,HPAI Strain,Bird Species,WOAH Classification,Sampling Method,Submitting Agency
0,North Dakota,Cass,9/12/2025,9/19/2025,EA H5,Canada goose,Wild bird,Morbidity/Mortality,ND Game and Fish
1,Pennsylvania,Bucks,9/8/2025,9/19/2025,EA H5,Black vulture,Wild bird,Morbidity/Mortality,PA Game Commission
2,Pennsylvania,Delaware,9/4/2025,9/19/2025,EA H5,Black vulture,Wild bird,Morbidity/Mortality,PA Game Commission


## Define Regions

We'll need to define regions for geographic analysis.

In [42]:
import us

region_map = {
    'Northeast': ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA'],
    'Midwest': ['IL', 'IN', 'MI', 'OH', 'WI', 'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD'],
    'South': ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'DC', 'WV', 'AL', 'KY', 'MS', 'TN', 'AR', 'LA', 'OK', 'TX'],
    'West': ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT', 'WY', 'AK', 'CA', 'HI', 'OR', 'WA']
}

state_to_region = {}
for region, states in region_map.items():
    for state in states:
        state_to_region[state] = region

df1 = hpai_data.copy()
df1['State'] = df1['State'].apply(lambda x: us.states.lookup(x).abbr if us.states.lookup(x) else x)
df1['Region'] = df1['State'].map(state_to_region)

df1.head(n=5)

,State,County,Collection Date,Date Detected,HPAI Strain,Bird Species,WOAH Classification,Sampling Method,Submitting Agency,Region
0,ND,Cass,9/12/2025,9/19/2025,EA H5,Canada goose,Wild bird,Morbidity/Mortality,ND Game and Fish,Midwest
1,PA,Bucks,9/8/2025,9/19/2025,EA H5,Black vulture,Wild bird,Morbidity/Mortality,PA Game Commission,Northeast
2,PA,Delaware,9/4/2025,9/19/2025,EA H5,Black vulture,Wild bird,Morbidity/Mortality,PA Game Commission,Northeast
3,NJ,Warren,9/11/2025,9/19/2025,EA H5,Black vulture,Wild bird,Morbidity/Mortality,NJ DEP,Northeast
4,NJ,Warren,9/11/2025,9/19/2025,EA H5,Black vulture,Wild bird,Morbidity/Mortality,NJ DEP,Northeast


## Define Seasons

We'll need to define seasons for temporal analysis. We use meteorological seasons:
- Winter: December, January, February
- Spring: March, April, May
- Summer: June, July, August
- Fall: September, October, November

In [43]:
seasons_map = {
    'Winter': ['Dec', 'Jan', 'Feb'],
    'Spring': ['Mar', 'Apr', 'May'],
    'Summer': ['Jun', 'Jul', 'Aug'],
    'Fall': ['Sep', 'Oct', 'Nov']
}

def parse_date(s):
    '''
    Attempts to parse a date string of format 'MM/DD/YYYY' to a datetime.

    If parsing fails, it returns pd.NaT (Not a Time).
    '''

    try:
        return pd.to_datetime(s, format='%m/%d/%Y', errors='coerce')
    except:
        return pd.NaT

month_to_season = {}
for season, months in seasons_map.items():
    for month in months:
        month_to_season[month] = season

df2 = df1.copy()

# convert date strings to datetime objects
# if parsing fails, the value will be set to pd.NaT because sometimes the date strings are invalid
df2['Date Detected'] = df2['Date Detected'].apply(parse_date)
df2['Collection Date'] = df2['Collection Date'].apply(parse_date)

# extract detection month and year
df2['Detection Month'] = pd.to_datetime(df2['Date Detected']).dt.strftime('%b')
df2['Detection Year'] = pd.to_datetime(df2['Date Detected']).dt.strftime('%Y')
df2 = df2.drop(columns=['Date Detected'])

# also extract collection month and year
df2['Collection Month'] = pd.to_datetime(df2['Collection Date']).dt.strftime('%b')
df2['Collection Year'] = pd.to_datetime(df2['Collection Date']).dt.strftime('%Y')
df2 = df2.drop(columns=['Collection Date'])

# map detection month to season
# use detection month because that's when the case is actually confirmed
df2['Season'] = df2['Detection Month'].map(month_to_season)

df2.head(n=1)

,State,County,HPAI Strain,Bird Species,WOAH Classification,Sampling Method,Submitting Agency,Region,Detection Month,Detection Year,Collection Month,Collection Year,Season
0,ND,Cass,EA H5,Canada goose,Wild bird,Morbidity/Mortality,ND Game and Fish,Midwest,Sep,2025,Sep,2025,Fall


## Question 1 - How does the frequency with which agencies collect samples affect the number of HPAI cases detected?

Goal is to see if there's any bias in the data based on when and where agencies collect samples, and how often they do so. Some agencies may collect more samples in a certain region because of proximity or season because of staffing (among other reasons), which could skew the data.

Let's see which agencies collect the most samples. We'll look at the number of samples collected by each agency across each year and see if there's a pattern.


In [71]:
df3 = df2.copy()
df3 = df3.loc[df3['Collection Year'].isin(['2022','2023','2024','2025'])]

df3 = df3.groupby(['Submitting Agency', 'Collection Year']).size().reset_index(name='Samples Collected').sort_values(by=['Samples Collected'], ascending=False)
df3_pivot = df3.pivot_table(index='Submitting Agency', columns='Collection Year', values='Samples Collected', fill_value=0).sort_values(by=['2022','2023','2024','2025'], ascending=False)

agency_totals = df3_pivot.sum(axis=1)
year_totals = df3_pivot.sum(axis=0)
df3_pivot['Total Samples'] = df3_pivot.sum(axis=1)
df3_pivot['% of All Samples'] = round(agency_totals / year_totals.sum() * 100.0, ndigits=3)

df3_pivot.head(n=10)

Collection Year,2022,2023,2024,2025,Total Samples,% of All Samples
Submitting Agency,,,,,,
NWDP,2939.0,1949.0,1950.0,916.0,7754.0,54.266
Private (non-government) submission,833.0,191.0,303.0,91.0,1418.0,9.924
SCWDS,256.0,90.0,65.0,4.0,415.0,2.904
CA DFW/CAHFS,218.0,113.0,72.0,40.0,443.0,3.100
NY DEC,159.0,66.0,77.0,145.0,447.0,3.128
MI DNR,145.0,14.0,6.0,81.0,246.0,1.722
UT DWR,112.0,33.0,5.0,1.0,151.0,1.057
FL FWCC,112.0,4.0,39.0,5.0,160.0,1.120
OR DFW,101.0,50.0,36.0,2.0,189.0,1.323


For the sake of brevity (there are 126 agencies), this is just the top 10.

The NWDP (National Wildlife Disease Program) is the agency that collects the most samples by far. See https://www.aphis.usda.gov/national-wildlife-programs/nwdp for more info on the NWDP. Private entities collect the next most samples.

Let's drill down into how many samples each agency collects during the seasons.

In [70]:
df4 = df2.copy()
df4 = df4.loc[df4['Collection Year'].isin(['2022','2023','2024','2025'])]

df4 = df4.groupby(['Submitting Agency', 'Collection Year', 'Season']).size().reset_index(name='Samples Collected').sort_values(by=['Samples Collected'], ascending=False)

df4_pivot = df4.pivot_table(index=['Submitting Agency', 'Season'], columns='Collection Year', values='Samples Collected', fill_value=0)

agency_totals = df4_pivot.sum(axis=1)
year_totals = df4_pivot.sum(axis=0)
df4_pivot['Total Samples'] = df4_pivot.sum(axis=1)
df4_pivot['% of Agency Samples'] = round(agency_totals / df3_pivot['Total Samples'] * 100.0, ndigits=3)
df4_pivot['% of All Samples'] = round(agency_totals / year_totals.sum() * 100.0, ndigits=3)

df4_pivot = df4_pivot.sort_values(by=['% of All Samples'], ascending=False)
df4_pivot.head(n=10)

Collection Year                               2022    2023    2024   2025  \
Submitting Agency                   Season                                  
NWDP                                Winter   785.0   511.0  1429.0  637.0   
                                    Fall    1449.0  1162.0   335.0   87.0   
                                    Spring   610.0    43.0    75.0  143.0   
Private (non-government) submission Spring   350.0    41.0   101.0   11.0   
NWDP                                Summer    95.0   233.0   111.0   49.0   
Private (non-government) submission Fall     236.0    58.0    35.0    3.0   
                                    Summer   101.0     7.0   127.0   65.0   
Tufts University                    Summer    64.0     0.0    18.0  209.0   
Private (non-government) submission Winter   146.0    85.0    40.0   12.0   
CA DFW/CAHFS                        Winter    94.0    58.0    33.0    0.0   

Collection Year                             Total Samples  \
Submitting Agency                   Season                  
NWDP                                Winter         3362.0   
                                    Fall           3033.0   
                                    Spring          871.0   
Private (non-government) submission Spring          503.0   
NWDP                                Summer          488.0   
Private (non-government) submission Fall            332.0   
                                    Summer          300.0   
Tufts University                    Summer          291.0   
Private (non-government) submission Winter          283.0   
CA DFW/CAHFS                        Winter          185.0   

Collection Year                             % of Agency Samples  \
Submitting Agency                   Season                        
NWDP                                Winter               43.358   
                                    Fall                 39.115   
                                    Spring               11.233   
Private (non-government) submission Spring               35.472   
NWDP                                Summer                6.294   
Private (non-government) submission Fall                 23.413   
                                    Summer               21.157   
Tufts University                    Summer               85.088   
Private (non-government) submission Winter               19.958   
CA DFW/CAHFS                        Winter               41.761   

Collection Year                             % of All Samples  
Submitting Agency                   Season                    
NWDP                                Winter            23.529  
                                    Fall              21.226  
                                    Spring             6.096  
Private (non-government) submission Spring             3.520  
NWDP                                Summer             3.415  
Private (non-government) submission Fall               2.323  
                                    Summer             2.100  
Tufts University                    Summer             2.037  
Private (non-government) submission Winter             1.981  
CA DFW/CAHFS                        Winter             1.295

Top 10 for sake of brevity.

Still notice that the NWDP collects the most samples. Interestingly, the number of samples collected by the NWDP seems to peak in the winter and fall seasons regardless of year. Overall, they collect the bulk of their samples in the fall and winter seasons, and account for a little over 50% of all samples collected.

Finally, let's look at where agencies collect the most samples.

In [69]:
df5 = df2.copy()
df5 = df5.loc[df5['Collection Year'].isin(['2022','2023','2024','2025'])]
df5 = df5.groupby(['Submitting Agency', 'Collection Year', 'Region']).size().reset_index(name='Samples Collected').sort_values(by=['Samples Collected'], ascending=False)

df5_pivot = df5.pivot_table(index=['Submitting Agency', 'Region'], columns='Collection Year', values='Samples Collected', fill_value=0)

agency_totals = df5_pivot.sum(axis=1)
year_totals = df5_pivot.sum(axis=0)
df5_pivot['Total Samples'] = df5_pivot.sum(axis=1)
df5_pivot['% of Agency Samples'] = round(agency_totals / df3_pivot['Total Samples'] * 100.0, ndigits=3)
df5_pivot['% of All Samples'] = round(agency_totals / year_totals.sum() * 100.0, ndigits=3)

df5_pivot = df5_pivot.sort_values(by=['% of All Samples'], ascending=False)
df5_pivot.head(n=15)


Collection Year                                  2022   2023   2024   2025  \
Submitting Agency                   Region                                   
NWDP                                Midwest    1074.0  731.0  364.0  205.0   
                                    South       842.0  512.0  708.0  155.0   
                                    West        626.0  600.0  678.0  115.0   
                                    Northeast   397.0  106.0  200.0  441.0   
NY DEC                              Northeast   159.0   66.0   76.0  145.0   
CA DFW/CAHFS                        West        218.0  113.0   72.0   40.0   
Private (non-government) submission Midwest     333.0   27.0   48.0   24.0   
                                    South       271.0   34.0   81.0   23.0   
Tufts University                    Northeast    78.0   14.0   40.0  207.0   
Private (non-government) submission West        152.0   92.0   51.0   14.0   
                                    Northeast    77.0   38.0  123.0   30.0   
MI DNR                              Midwest     145.0   14.0    6.0   81.0   
SCWDS                               South       124.0   53.0   63.0    3.0   
PA Game Commission                  Northeast    74.0   26.0   13.0  122.0   
OR DFW                              West        101.0   50.0   36.0    2.0   

Collection Year                                Total Samples  \
Submitting Agency                   Region                     
NWDP                                Midwest           2374.0   
                                    South             2217.0   
                                    West              2019.0   
                                    Northeast         1144.0   
NY DEC                              Northeast          446.0   
CA DFW/CAHFS                        West               443.0   
Private (non-government) submission Midwest            432.0   
                                    South              409.0   
Tufts University                    Northeast          339.0   
Private (non-government) submission West               309.0   
                                    Northeast          268.0   
MI DNR                              Midwest            246.0   
SCWDS                               South              243.0   
PA Game Commission                  Northeast          235.0   
OR DFW                              West               189.0   

Collection Year                                % of Agency Samples  \
Submitting Agency                   Region                           
NWDP                                Midwest                 30.616   
                                    South                   28.592   
                                    West                    26.038   
                                    Northeast               14.754   
NY DEC                              Northeast               99.776   
CA DFW/CAHFS                        West                   100.000   
Private (non-government) submission Midwest                 30.465   
                                    South                   28.843   
Tufts University                    Northeast               99.123   
Private (non-government) submission West                    21.791   
                                    Northeast               18.900   
MI DNR                              Midwest                100.000   
SCWDS                               South                   58.554   
PA Game Commission                  Northeast              100.000   
OR DFW                              West                   100.000   

Collection Year                                % of All Samples  
Submitting Agency                   Region                       
NWDP                                Midwest              16.614  
                                    South                15.515  
                                    West                 14.130  
                                    Northeast             8.006

Some key observations from all of the above:
- NWDP generally collects the most samples across all years, seasons, and regions.
    - Largest percentage too (>50%)
- NWDP seems to collect samples most frequently in the winter and fall seasons.
- NWDP collects significantly fewer samples in the Northeast region but does collect a similar (higher) number of samples in the Midwest, South, and West regions.
    - 2025 is an exception in that the NWDP so far has collected more samples in the Northeast than any other region.

These observations suggest the following:
1. NWDP has the most impact on detections overall.
2. There may be seasonal bias in the data due to the NWDP's collection patterns.
3. There may be regional bias in the data due to the NWDP's smaller number of samples collected in the Northeast region.

## Question 2 - Is there a correlation between geographic region and season, and number of HPAI cases?

Goal is to assess how region and season impact number of HPAI cases, if at all.

Let's see which regions have the most cases overall across all years.

In [67]:
df6 = df2.copy()
df6 = df6.groupby(['Region', 'Detection Year']).size().reset_index(name='HPAI Cases').sort_values(by=['HPAI Cases'], ascending=False)
df6_pivot = df6.pivot_table(index='Region', columns='Detection Year', values='HPAI Cases', fill_value=0)
region_totals = df6_pivot.sum(axis=1)
year_totals = df6_pivot.sum(axis=0)
df6_pivot['Total Cases'] = df6_pivot.sum(axis=1)
df6_pivot['% of All Cases'] = round(region_totals / year_totals.sum() * 100.0, ndigits=3)
df6_pivot

Detection Year,2022,2023,2024,2025,Total Cases,% of All Cases
Region,,,,,,
Midwest,2039.0,859.0,567.0,637.0,4102.0,28.296
Northeast,800.0,273.0,471.0,1092.0,2636.0,18.183
South,1463.0,582.0,990.0,492.0,3527.0,24.329
West,1619.0,1083.0,1026.0,504.0,4232.0,29.192


Recall previously that the NWDP collects the most samples overall, and specifically doesn't collect as many samples in the Northeast region. This may explain why the Northeast has the fewest cases overall by a significant margin.

Let's see how many cases each region has during the seasons. From our observations, we would expect the winter and fall seasons to have the most cases.

In [ ]:
df7 = df2.copy()
df7 = df7.loc[df7['Detection Year'].isin(['2022','2023','2024','2025'])]

df7 = df7.groupby(['Region', 'Detection Year', 'Season']).size().reset_index(name='HPAI Cases').sort_values(by=['HPAI Cases'], ascending=False)
df7_pivot = df7.pivot_table(index=['Region', 'Season'], columns='Detection Year', values='HPAI Cases', fill_value=0)

region_and_season_totals = df7_pivot.sum(axis=1)
year_totals = df7_pivot.sum(axis=0)
df7_pivot['Total Cases'] = region_and_season_totals
df7_pivot['% of Regional Cases'] = round(df7_pivot['Total Cases'] / df6_pivot['Total Cases'] * 100.0, ndigits=3)
df7_pivot['% of All Cases'] = round(region_and_season_totals / year_totals.sum() * 100.0, ndigits=3)

df7_pivot

Detection Year      2022   2023   2024   2025  Total Cases  \
Region    Season                                             
Midwest   Fall     688.0  510.0  132.0   67.0       1397.0   
          Spring  1039.0   82.0   47.0   99.0       1267.0   
          Summer   137.0   14.0  143.0  290.0        584.0   
          Winter   175.0  253.0  245.0  181.0        854.0   
Northeast Fall     308.0   22.0   35.0   16.0        381.0   
          Spring   165.0   91.0  153.0   89.0        498.0   
          Summer   140.0    4.0  107.0  580.0        831.0   
          Winter   187.0  156.0  176.0  407.0        926.0   
South     Fall     586.0   48.0  333.0    5.0        972.0   
          Spring   209.0   71.0   50.0   48.0        378.0   
          Summer   119.0   40.0   72.0  162.0        393.0   
          Winter   549.0  423.0  535.0  277.0       1784.0   
West      Fall     894.0  458.0  345.0   35.0       1732.0   
          Spring   244.0  145.0   52.0   52.0        493.0   
          Summer   246.0   71.0  159.0  252.0        728.0   
          Winter   235.0  409.0  470.0  165.0       1279.0   

Detection Year    % of Regional Cases  % of All Cases  
Region    Season                                       
Midwest   Fall                 34.057           9.636  
          Spring               30.887           8.740  
          Summer               14.237           4.028  
          Winter               20.819           5.891  
Northeast Fall                 14.454           2.628  
          Spring               18.892           3.435  
          Summer               31.525           5.732  
          Winter               35.129           6.388  
South     Fall                 27.559           6.705  
          Spring               10.717           2.607  
          Summer               11.143           2.711  
          Winter               50.581          12.306  
West      Fall                 40.926          11.947  
          Spring               11.649           3.401  
          Summer               17.202           5.022  
          Winter               30.222           8.823

Some observations:
- In the Midwest, most cases were detected in the fall and spring seasons.
- In the Northeast, most cases were detected in the summer and winter.
- In the South, most cases were detected in the fall and winter.
- In the West, most cases were detected in the fall and winter.

As expected, the Northeast has the smallest percentage of cases by a significant margin. This aligns with our earlier observations about the NWDP's collection patterns, which indicated a lower focus on this region. The NWDP is much more active in the other regions, which may help explain why those regions have a higher percentage of cases, among other factors.

It's important to note that there was an outbreak of HPAI in 2022, specifically in a commercial flock in Indiana, which is in the Midwest region. This outbreak likely contributed to the higher number of cases observed in the Midwest during that year. See https://www.congress.gov/crs-product/R48518 for more information on the initial outbreak.
